<a href="https://colab.research.google.com/github/raheelam98/LangChain_Fundamentals/blob/main/langchain_course_jupyter/section_3_data_processing/part6_output_parser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Output Parser**

[Pydantic parser](https://python.langchain.com/v0.1/docs/modules/model_io/output_parsers/types/pydantic/)

In [28]:
# Install the required packages:
%%capture --no-stderr
%pip install -U  langsmith # check
%pip install --quiet -U  langchain_google_genai langchain_core langchain langgraph   python-dotenv langsmith

In [29]:
%%capture --no-stderr
%pip install ChatGoogleGenerativeAI

In [30]:
# packages
%%capture --no-stderr
%pip install -U tavily-python langchain_community

In [31]:
import os
from google.colab import userdata

os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGCHAIN_API_KEY')

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "ice_breaker"

In [44]:
# API Keys
# Get the GEMINI API key from user data
from google.colab import userdata
gemini_api_key = userdata.get('GEMINI_API_KEY')

In [ ]:
# from langchain_google_genai import ChatGoogleGenerativeAI

# llm: ChatGoogleGenerativeAI = ChatGoogleGenerativeAI(model="gemini-1.5-flash", api_key=gemini_api_key, temperature=0)

In [45]:
# llm
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    max_retries=2,
    api_key=gemini_api_key
)

In [34]:
llm.invoke("hi")

AIMessage(content='Hi there! How can I help you today?\n', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-61029d36-b3c9-41f1-8481-34d08118e012-0', usage_metadata={'input_tokens': 2, 'output_tokens': 11, 'total_tokens': 13, 'input_token_details': {'cache_read': 0}})

In [ ]:
# # Restart Runtime and Re-authenticate
# from google.colab import auth
# auth.authenticate_user()


In [35]:
result = llm.invoke("hello")
print(result)

content='Hello there! How can I help you today?\n' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []} id='run-cbf7369c-2557-4fe2-8ee4-7756c5af04f2-0' usage_metadata={'input_tokens': 2, 'output_tokens': 11, 'total_tokens': 13, 'input_token_details': {'cache_read': 0}}


In [ ]:
# from langchain_google_genai import ChatGoogleGenerativeAI
# model: ChatGoogleGenerativeAI = ChatGoogleGenerativeAI(model = "gemini-1.5-flash")
# model.invoke("hi")

[Tavily Search](https://python.langchain.com/docs/integrations/tools/tavily_search/)

[TAVILY_API_KEY](https://tavily.com/)


In [46]:
# API keys set up
os.environ["TAVILY_API_KEY"] = userdata.get("TAVILY_API_KEY")

[hwchase17/react](https://smith.langchain.com/hub/hwchase17/react)



**Create React Agent**

In [48]:
# app/tools/tools.py
from langchain_community.tools.tavily_search import TavilySearchResults


def get_profile_url_tavily(name: str):
    """Searches for Linkedin or Twitter Profile Page."""
    search = TavilySearchResults()
    res = search.run(f"{name}")
    return res

In [39]:
# app/agents/linkedin_lookup_agent.py

from langchain_core.prompts import PromptTemplate
from langchain_core.tools import Tool

from langchain.agents import (create_react_agent, AgentExecutor)
from langchain import hub
# from app.tools.tool import get_profile_url_tavily

# from langchain_community.tools.tavily_search import TavilySearchResults

# from dotenv import load_dotenv
# from langchain_google_genai import ChatGoogleGenerativeAI
# from pydantic import  SecretStr
# import os

# load_dotenv()

# gemini_api_key = SecretStr(os.environ["GEMINI_API_KEY"])

# if not gemini_api_key:
#     raise ValueError("GEMINI_API_KEY is not set. Please provide the API key in your environment.")

# print("API Key:", gemini_api_key)

# # Initialize the ChatGoogleGenerativeAI model
# llm = ChatGoogleGenerativeAI(
#     model="gemini-1.5-flash",  # Specify the model
#     api_key=gemini_api_key,  # Pass the API key
#     )



# def get_profile_url_tavily(name: str):
#     """Search for LinkedIn or Twitter Profile Page."""
#     search = TavilySearchResults()
#     res = search.run(f"{name}")
#     return res


def lookup_linkedin(name: str):
    # llm = ChatGoogleGenerativeAI(
    #     model="gemini-1.5-flash"
    # )
    template = """give the full name {name_of_person} I want you to get it me a link to their Linkedin profile page.
                  Your answer should contain only a URL"""

    prompt_template = PromptTemplate(
        template=template, input_variables=["name_of_person"]
    )

    tools_for_agent = [
        Tool(
            name="Crawl Google 4 linkedin profile page",
            func=get_profile_url_tavily,
            description="useful for when you need get the Linkedin Page URL"
        )
    ]

    react_prompt = hub.pull("hwchase17/react")
    agent = create_react_agent(
        llm=llm,
        tools=tools_for_agent,
        prompt=react_prompt
    )

    agent_executor = AgentExecutor(
        agent=agent,
        tools=tools_for_agent,
        verbose=True
    )

    result = agent_executor.invoke(
        input={"input": prompt_template.format_prompt(name_of_person=name)}
    )

    linkedin_profile_url = result["output"]
    return linkedin_profile_url


# if __name__ == '__main__':
#     print("hello LangChain linkedin lookup agent")
#     linkedin_profile_url = lookup_linkedin(name="Eden Marco Udemy")
#     print(linkedin_profile_url)



In [41]:
# # Example function call
print("Linkedin Lookup\n\n")

linkedin_profile_url = lookup_linkedin(name="Elon Musk")
print(linkedin_profile_url)

Linkedin Lookup




> Entering new AgentExecutor chain...
Thought: I need to find Elon Musk's LinkedIn profile URL.  I can use Google to search for it.

Action: Crawl Google 4 linkedin profile page

Action Input: name='Elon Musk'
[{'url': 'https://www.mountbonnell.info/elons-austin/unmasking-elon-musk-the-shocking-truth-behind-his-real-name-revealed', 'content': '7 days ago ... Elon Musk, the renowned entrepreneur and CEO of Tesla and SpaceX, is a name that has become synonymous with innovation and ambition.'}, {'url': 'https://www.youtube.com/watch?v=YJxdh5-Za4Q', 'content': '3 days ago ... Tech billionaire Elon Musk has updated his X, formerly known as Twitter, and changed his name from "Elon Musk" to "Kekius Maximus".'}, {'url': 'https://www.hindustantimes.com/world-news/us-news/elon-musk-changes-his-name-on-x-again-after-kekius-maximus-this-time-its-101735814599604.html', 'content': '2 days ago ... Elon Musk has once again changed his name on X. On Tuesday, the Tesla CEO debuted “K

In [51]:
# app/agents/twitter_lookup_agent.py

from dotenv import load_dotenv

load_dotenv()
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts.prompt import PromptTemplate
from langchain_core.tools import Tool
from langchain.agents import (
    create_react_agent,
    AgentExecutor,
)
from langchain import hub
# from tools.tools import get_profile_url_tavily


def lookup_twitter(name: str) -> str:
    # llm = ChatOpenAI(
    #     temperature=0,
    #     model_name="gpt-4o-mini",
    # )
    template = """
       given the name {name_of_person} I want you to find a link to their Twitter profile page, and extract from it their username
       In Your Final answer only the person's username"""
    prompt_template = PromptTemplate(
        template=template, input_variables=["name_of_person"]
    )
    tools_for_agent = [
        Tool(
            name="Crawl Google 4 Twitter profile page",
            func=get_profile_url_tavily,
            description="useful for when you need get the Twitter Page URL",
        )
    ]

    react_prompt = hub.pull("hwchase17/react")
    agent = create_react_agent(llm=llm, tools=tools_for_agent, prompt=react_prompt)
    agent_executor = AgentExecutor(agent=agent, tools=tools_for_agent, verbose=True)

    result = agent_executor.invoke(
        input={"input": prompt_template.format_prompt(name_of_person=name)}
    )

    twitter_username = result["output"]
    return twitter_username


# if __name__ == "__main__":
#     print(lookup(name="Elon Musk"))

In [52]:
# # Example function call
print("Twitter Lookup\n\n")
twitter_lookup = lookup_twitter(name="Elon Musk")
print("Twitter Lookup:", twitter_lookup)

Twitter Lookup




> Entering new AgentExecutor chain...
Thought: I need to find Elon Musk's Twitter profile URL using a search engine, then extract the username from that URL.  I'll use the provided tool to crawl Google for the Twitter profile page.

Action: Crawl Google 4 Twitter profile page
Action Input: "Elon Musk"
[{'url': 'https://www.tesla.com/elon-musk', 'content': "As the co-founder and CEO of Tesla, Elon leads all product design, engineering and global manufacturing of the company's electric vehicles, battery products"}, {'url': 'https://www.bbc.com/news/articles/cy53vz1qpx1o', 'content': 'Elon Musk changes his name to Kekius Maximus on X Elon Musk changes his name to Kekius Maximus on X The world\'s richest man, Elon Musk, has sparked speculation after changing his name on his social media platform X to "Kekius Maximus". Elon Musk\'s \'social experiment on humanity\': How X evolved in 2024 ------------------------------------------------------------------ Elon Musk\'s \'soc

### output_parsers

In [53]:
# app/output_parsers.py

from typing import List, Dict, Any

from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

class Summary(BaseModel):
    summary: str = Field(description="summary")
    facts: List[str] = Field(description="interesting facts about them")

    def to_dict(self) -> Dict[str, Any]:
        return {"summary": self.summary, "facts": self.facts}

summary_parser = PydanticOutputParser(pydantic_object=Summary)

In [57]:
# app/ice_breaker.py

from langchain.prompts.prompt import PromptTemplate


# from third_parties.linkedin import scrape_linkedin_profile
# from agents.linkedin_lookup_agent import lookup as linkedin_lookup_agent

# from dotenv import load_dotenv
# from langchain_google_genai import ChatGoogleGenerativeAI
# from pydantic import  SecretStr
# import os

# load_dotenv()

# gemini_api_key = SecretStr(os.environ["GEMINI_API_KEY"])

# if not gemini_api_key:
#     raise ValueError("GEMINI_API_KEY is not set. Please provide the API key in your environment.")

# print("API Key:", gemini_api_key)

# # Initialize the ChatGoogleGenerativeAI model
# llm = ChatGoogleGenerativeAI(
#     model="gemini-1.5-flash",  # Specify the model
#     api_key=gemini_api_key,  # Pass the API key
#     )


def ice_break_with(name: str) -> str:

    # linkedin_username = linkedin_profile_url(name=name)
    # linkedin_data = scrape_linkedin_profile(linkedin_profile_url=linkedin_username)

    twitter_username = lookup_twitter(name=name)
    # tweets = scrape_user_tweets(username=twitter_username)

    summary_template = """
    given the Linkedin information {information}, and their lates twiter {twitter_post} I want you to create:
    1. A short summary
    2. two interesting facts about them

    Use both information from twitter and Linkedin
    \n{format_instructions}
    """

    summary_prompt_template = PromptTemplate(
        input_variables=["information", "twitter_posts"],
        template=summary_template,
        partial_variables={
            "format_instructions": summary_parser.get_format_instructions()
        }
    )

    # llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")

    chain = summary_prompt_template | llm | summary_parser

    # res = chain.invoke(input={"information": linkedin_data, "twitter_posts": tweets})
    res = chain.invoke(input={"twitter_posts": twitter_username})

    print(res)


In [58]:
# # Example function call
print("Twitter tweet ")
twitter_tweets = lookup_twitter(name="Harrison Chase")
print("Twitter Lookup:", twitter_tweets)

Twitter tweet 


> Entering new AgentExecutor chain...
Thought: I need to find Harrison Chase's Twitter profile URL to extract their username.  I can use the "Crawl Google 4 Twitter profile page" tool for this.

Action: Crawl Google 4 Twitter profile page
Action Input: Harrison Chase
[{'url': 'https://www.bigdatawire.com/people-to-watch-2024-harrison-chase/', 'content': 'People to Watch 2024 - Harrison Chase - BigDATAwire People to Watch 2024 – Harrison Chase Harrison Chase is the co-founder and CEO of LangChain, developer of the eponymous tool for chaining together interoperable components for GenAI applications. This website uses cookies to improve your experience. Privacy & Cookies Policy Out of these, the cookies that are categorized as necessary are stored on your browser as they are essential for the working of basic functionalities of the website. Necessary cookies are absolutely essential for the website to function properly. Any cookies that may not be particularly necessary f

In [59]:
# # Example function call
print("ice breaker\n\n ")
ice_breaker  = ice_break_with(name="Harrison Chase")
print("ice breker:", ice_breaker)

ice breaker

 


> Entering new AgentExecutor chain...
Thought: I need to find Harrison Chase's Twitter profile URL to extract his username.  I can use the "Crawl Google 4 Twitter profile page" action for this.

Action: Crawl Google 4 Twitter profile page
Action Input: Harrison Chase
[{'url': 'https://www.bigdatawire.com/people-to-watch-2024-harrison-chase/', 'content': 'People to Watch 2024 - Harrison Chase - BigDATAwire People to Watch 2024 – Harrison Chase Harrison Chase is the co-founder and CEO of LangChain, developer of the eponymous tool for chaining together interoperable components for GenAI applications. This website uses cookies to improve your experience. Privacy & Cookies Policy Out of these, the cookies that are categorized as necessary are stored on your browser as they are essential for the working of basic functionalities of the website. Necessary cookies are absolutely essential for the website to function properly. Any cookies that may not be particularly necessary f

KeyError: "Input to PromptTemplate is missing variables {'twitter_post', 'information'}.  Expected: ['information', 'twitter_post'] Received: ['twitter_posts']\nNote: if you intended {twitter_post} to be part of the string and not a variable, please escape it with double curly braces like: '{{twitter_post}}'.\nFor troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_PROMPT_INPUT "